In [1]:
#imports
#from jupyter_plotly_dash import JupyterDash
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import plotly
import pandas as pd

#import stylesheet
external_stylesheets = ['https://codepen.io/amyoshino/pen/jzXypZ.css']

In [2]:
df = pd.read_csv('zipcode-data.csv')

In [3]:
df

,Zip Code,Area Name,percent no internet,Mean Income Past 12 Months
0,38126,South Forum - Washington Heights,0.8308,26346
1,38106,South Memphis,0.8223,31403
2,38108,Hollywood - Hyde Park - Nutbush,0.7356,31278
3,38114,Rozelle - Castalia - Orange Mound - Fairhills ...,0.7064,38428
4,38105,Uptown - Pinch District,0.6982,36980
5,38127,Frayser,0.6283,37886
6,38109,Coro Lake - White Haven,0.6271,42657
7,38107,North Memphis - Snowden - New Chicago,0.5746,40915
8,38116,Whitehaven,0.5745,52128
9,38118,Oakville - Parkway Village,0.5271,42729


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 4 columns):
Zip Code                      11 non-null int64
Area Name                     11 non-null object
percent no internet           11 non-null float64
Mean Income Past 12 Months    11 non-null int64
dtypes: float64(1), int64(2), object(1)
memory usage: 432.0+ bytes


In [5]:
df['Zip Code'] = df['Zip Code'].astype(str)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 4 columns):
Zip Code                      11 non-null object
Area Name                     11 non-null object
percent no internet           11 non-null float64
Mean Income Past 12 Months    11 non-null int64
dtypes: float64(1), int64(1), object(2)
memory usage: 432.0+ bytes


In [7]:
###### display function to render dashboard directly to notebook
from IPython import display
import os
def show_app(app, port = 9999, 
             width = 700, 
             height = 350, 
             offline = False,
            in_binder = None):
    in_binder ='JUPYTERHUB_SERVICE_PREFIX' in os.environ if in_binder is None else in_binder
    if in_binder:
        base_prefix = '{}proxy/{}/'.format(os.environ['JUPYTERHUB_SERVICE_PREFIX'], port)
        url = 'https://hub.mybinder.org{}'.format(base_prefix)
        app.config.requests_pathname_prefix = base_prefix
    else:
        url = 'http://localhost:%d' % port
        
    iframe = '<a href="{url}" target="_new">Open in new window</a><hr><iframe src="{url}" width={width} height={height}></iframe>'.format(url = url, 
                                                                                  width = width, 
                                                                                  height = height)
    
    display.display_html(iframe, raw = True)
    if offline:
        app.css.config.serve_locally = True
        app.scripts.config.serve_locally = True
    return app.run_server(debug=False, # needs to be false in Jupyter
                          host = '0.0.0.0',
                          port=port)

In [8]:
app = dash.Dash(__name__, external_stylesheets = external_stylesheets)

In [ ]:
city_data = {
    '38126': {'x': ['South Forum'], 'y': [.8308]},
    '38106': {'x': ['South Memphis'], 'y': [.8223]},
    '38108': {'x': ['Hollywood'], 'y': [.7356]},
    '38114': {'x': ['Orange Mound'], 'y': [.7064]},
    '38105': {'x': ['Pinch'], 'y': [.6982]},
    '38127': {'x': ['Frayser'], 'y': [.6283]},
    '38109': {'x': ['Coro lake'], 'y': [.6271]},
    '38107': {'x': ['North Memphis'], 'y': [.5746]},
    '38116': {'x': ['Whitehaven'], 'y': [.5745]},
    '38118': {'x': ['Oakville'], 'y': [.5271]},
    '38115': {'x': ['Hickory Ridge'], 'y': [.5175]}
}

In [ ]:
colors = {
    'background': '#000000',
    'text': '#5d76a9',
    'label': '#f5b112'
}

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.Div([
        html.Div([
            html.H1(
            children='Memphis Zip Code Data',
            style={
                'textAlign': 'center',
                'color': colors['text'],
                'padding-top': 40
                }
            ),
            html.Div(children='Visualizing U.S. Census Bureau datapoints for various Memphis zip codes', style={
            'textAlign': 'center',
            'color': colors['text'],
            'padding-bottom': 40
            })
        ], className='row'),
###############################################################      
        html.Div([
            dcc.Dropdown(
                id='demo-dropdown',
                options=[{'label': i, 'value': i} for i in df['Zip Code']],
                value='null',
                multi=True
            ),
        html.Div(id='dd-output-container')]),
###############################################################        
        html.Div([
            html.Div([
                dcc.Graph(
                    id='graph1'
                )
            ], className='eight columns'
            )
        ], className='row')
    ], className='ten columns offset-by-one'),    
])

@app.callback(
    dash.dependencies.Output('dd-output-container', 'children'),
    [dash.dependencies.Input('demo-dropdown', 'value')])
def update_output(value):
    return 'You have selected "{}"'.format(value)

@app.callback(
    dash.dependencies.Output('graph1', 'figure'),
    [dash.dependencies.Input('demo-dropdown', 'value')])
def update_image_src(selector):
    data = []
    #filtered_data = df[df['Zip Code'] == zip_code]
    for city in selector:
        data.append({'x': city_data[city]['x'],
                     'y': city_data[city]['y'],
                     'type': 'bar',
                     'name': city})
    figure = {
        'data': data,
        'layout': {
            'title': 'Percent of homes lacking broadband internet',
            'xaxis' : dict(
                title='Area Name',
                titlefont=dict(
                family='Courier New, monospace',
                size=20,
                color='#7f7f7f'
            )),
            'yaxis' : dict(
                title='% No Internet',
                titlefont=dict(
                family='Helvetica, monospace',
                size=20,
                color='#7f7f7f',
                tickformat=',.0%',
                range=[0,1]
            ))
        }
    }
    return figure

In [ ]:
l = []
x = df[df['Zip Code'] == '38126']['Zip Code'][0]
l.append(x)
l

['38126']

In [ ]:
show_app(app)

Open in new window

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:9999/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Apr/2020 17:14:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2020 17:14:30] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2020 17:14:30] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2020 17:14:30] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2020 17:14:30] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.3.1&m=1576595950 HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2020 17:14:30] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.1&m=1576596177 HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2020 17:14:30] "GET /_dash-component-suites/dash_renderer/dash_renderer.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2020 17:

data = df[df['Zip Code'] == '38126']
new = []
new.append(dict(
            x=data['Area Name'][0],
            y=data['percent no internet'][0],
            text=data['Zip Code'][0],
            type= 'bar',
            name='38126'
        ))

data

new

zip_code = '38126'
#df.loc[df['Zip Code'] == 38126, 'percent no internet'].values[1]
df.loc[df['Zip Code'] == zip_code, 'percent no internet'][0]


colors = {
    'background': '#000000',
    'text': '#5d76a9',
    'label': '#f5b112'
}

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.Div([
        html.Div([
            html.H1(
            children='Memphis Zip Code Data',
            style={
                'textAlign': 'center',
                'color': colors['text'],
                'padding-top': 40
                }
            ),
            html.Div(children='Visualizing U.S. Census Bureau datapoints for various Memphis zip codes', style={
            'textAlign': 'center',
            'color': colors['text'],
            'padding-bottom': 40
            })
        ], className='row'),
###############################################################      
        html.Div([
            dcc.Dropdown(
                id='demo-dropdown',
                options=[{'label': i, 'value': i} for i in df['Zip Code']],
                value='null',
                multi=True
            ),
        html.Div(id='dd-output-container')]),
###############################################################        
        html.Div([
            html.Div([
                dcc.Graph(
                    id='graph1',
                    figure={
                        'data': [
                            {'x': df.iloc[0:4, 1], 'y': df['percent no internet'], 'type': 'bar', 'name': 'Internet'},
                        ],
                        'layout': {
                            'title': 'Percent of homes lacking broadband internet',
                            #'xaxis': 'Zip Code Area',
                            #'yaxis': 'Percent Without Internet',
                            'plot_bgcolor': colors['background'],
                            'paper_bgcolor': colors['background'],
                            'font': {
                                'color': colors['label']
                            }
                        }
                    }
                )
            ], className='eight columns'
            ),
            html.Div([
                dcc.Graph(
                    id='graph2',
                    figure={
                        'data': [
                            {'x': df.iloc[0:5, 1], 'y': df.iloc[0:5,3], 'type': 'bar', 'name': 'Income'},
                        ],
                        'layout': {
                            'title': 'Mean Home Income',
                            'plot_bgcolor': colors['background'],
                            'paper_bgcolor': colors['background'],
                            'font': {
                                'color': colors['label']
                            }
                        }
                    }
                )
            ], className='four columns')   
        ], className='row')
    ], className='ten columns offset-by-one'),    
])

@app.callback(
    dash.dependencies.Output('dd-output-container', 'children'),
    [dash.dependencies.Input('demo-dropdown', 'value')])
def update_output(value):
    return 'You have selected "{}"'.format(value)